# Local Feature Extractors

Train a set of CNNs handling a inputs (segments of mel-spectrograms) in different scale. 

These CNN models will be used as loacal feature extrators in music genre tagger.

We plan to train 5 different scales of CNNs. Their segment length are: 20, 30, 60, 120, 240.

All CNNs contain 5 conv layers. The "vertical" pooling size in each layer would be: `[2, 3, 2, 2, 4]`.

The horizontal pooling sizes are set according to their segment length:

Segment Length | Pooling sizes
- | -
20 | `[2, 2, 2, 2, 1]`
30 | `[2, 2, 2, 2, 1]`
60 | `[3, 2, 2, 2, 2]`
120 | `[4, 3, 2, 2, 2]`
240 | `[4, 4, 3, 2, 2]`
    

In [30]:
import os    
os.environ['THEANO_FLAGS'] = "floatX=float32,device=cuda,exception_verbosity=high"
import theano

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input, Dense, merge, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
from keras.utils.visualize_util import plot
from keras.models import load_model
import matplotlib.pyplot as plt
import pickle

## Define model

In [10]:
def gen_model(seg_length, pool_sizes_hori):
    '''
    Generate model with different scales.
    seg_length and pooling layer sizes are set adjusting to different scales.
    '''
    psh = [0] + pool_sizes_hori # padding at front for index alignment
    
    # input
    x = Input(shape=(1, 96, seg_length))

    # 1st conv layer
    conv1 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv1_{}'.format(seg_length))(x)
    conv1 = BatchNormalization(axis=1, mode=2, name='BN1_{}'.format(seg_length))(conv1)
    conv1 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv1)
    conv1 = MaxPooling2D(pool_size=(2, psh[1]))(conv1)

    # 2nd conv layer
    conv2 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv2_{}'.format(seg_length))(conv1)
    conv2 = BatchNormalization(axis=1, mode=2, name='BN2_{}'.format(seg_length))(conv2)
    conv2 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv2)
    conv2 = MaxPooling2D(pool_size=(3, psh[2]))(conv2)

    # 3rd conv layer
    conv3 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv3_{}'.format(seg_length))(conv2)
    conv3 = BatchNormalization(axis=1, mode=2, name='BN3_{}'.format(seg_length))(conv3)
    conv3 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv3)
    conv3 = MaxPooling2D(pool_size=(2, psh[3]))(conv3)

    # 4th conv layer
    conv4 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv4_{}'.format(seg_length))(conv3)
    conv4 = BatchNormalization(axis=1, mode=2, name='BN4_{}'.format(seg_length))(conv4)
    conv4 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv4)
    conv4 = MaxPooling2D(pool_size=(2, psh[4]))(conv4)

    # 5th conv layer
    conv5 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv5_{}'.format(seg_length))(conv4)
    conv5 = BatchNormalization(axis=1, mode=2, name='BN5_{}'.format(seg_length))(conv5)
    conv5 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv5)
    conv5 = MaxPooling2D(pool_size=(4, psh[5]))(conv5)

    # Flatten the output of last conv layer (conv5)
    conv5 = Flatten()(conv5)
    
    # output layer
    out = Dense(10, input_shape=[32], activation='softmax')(conv5)
    
    # define model
    model = Model(input=x, output=out)
    
    return model

In [31]:
def gen_model_3(seg_length, pool_sizes_hori):
    '''
    Generate model with different scales.
    seg_length and pooling layer sizes are set adjusting to different scales.
    '''
    psh = [0] + pool_sizes_hori # padding at front for index alignment
    
    # input
    x = Input(shape=(1, 96, seg_length))

    # 1st conv layer
    conv1 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv1_{}'.format(seg_length))(x)
#     conv1 = BatchNormalization(axis=1, mode=2, name='BN1_{}'.format(seg_length))(conv1)
    conv1 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv1)
    conv1 = MaxPooling2D(pool_size=(2, psh[1]))(conv1)
    conv1 = Dropout(0.2)(conv1)

    # 2nd conv layer
    conv2 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv2_{}'.format(seg_length))(conv1)
#     conv2 = BatchNormalization(axis=1, mode=2, name='BN2_{}'.format(seg_length))(conv2)
    conv2 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv2)
    conv2 = MaxPooling2D(pool_size=(3, psh[2]))(conv2)
    conv2 = Dropout(0.2)(conv2)
    
    # 3rd conv layer
    conv3 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv3_{}'.format(seg_length))(conv2)
#     conv3 = BatchNormalization(axis=1, mode=2, name='BN3_{}'.format(seg_length))(conv3)
    conv3 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv3)
    conv3 = MaxPooling2D(pool_size=(2, psh[3]))(conv3)
    conv3 = Dropout(0.2)(conv3)
    
    # 4th conv layer
    conv4 = Convolution2D(32, 3, 3, border_mode='same', init='he_normal', name='conv4_{}'.format(seg_length))(conv3)
#     conv4 = BatchNormalization(axis=1, mode=2, name='BN4_{}'.format(seg_length))(conv4)
    conv4 = keras.layers.advanced_activations.ELU(alpha=1.0)(conv4)
    conv4 = MaxPooling2D(pool_size=(2, psh[4]))(conv4)
    
    # Flatten the output of last conv layer (conv5)
    flat = Flatten()(conv4)
    
    # Dense layer # 1
    dense1 = Dense(64, activation='relu')(flat)
    dense1 = Dropout(0.2)(dense1)
    
    # output layer
    out = Dense(10, activation='softmax')(dense1)
    
    # define model
    model = Model(input=x, output=out)
    
    return model

## Data Augmentation Generator

In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10.0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

## Helper Cells

In [34]:
### Save Model (for continue training) ###
model.save('./models/cnn_{}.h5'.format(seg_length))  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

In [ ]:
### Load previously trained model ###
# model = load_model('./models/cnn_{}.h5'.format(seg_length))

In [35]:
### Save Training History (for plotting) ###
import pickle, datetime
now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
with open('./train_logs/train_his_{}_{}'.format(seg_length, now), 'wb') as file_pi:
    pickle.dump(model_his.history, file_pi)

In [36]:
### Save Model weights (for future transferr) ###
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
model.save_weights('./weights/local_cnn_4_{}.h5'.format(seg_length))

In [ ]:
### Load Model weights ###
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
model.load_weights('./weights/local_cnn_{}.h5'.format(seg_length), by_name=True)

In [ ]:
### Plot function ###
def plot_acc_loss(model_his, seg_length):
    '''
    Plot train/val acc & loss in model training.
    '''
    # Create path (if not exist) for saving plots
    if not os.path.exists('./figs/'):
        os.mkdir('./figs/')
    
    # summarize history for accuracy
    fig = plt.figure()
    plt.plot(model_his.history['acc'])
    plt.plot(model_his.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    fig.savefig('./figs/acc_{}.png'.format(seg_length))

    # summarize history for loss
    fig = plt.figure()
    plt.plot(model_his.history['loss'])
    plt.plot(model_his.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    fig.savefig('./figs/loss_{}.png'.format(seg_length))
    
    return

In [17]:
### Plot Current Training History ###
plot_acc_loss(model_his)

NameError: name 'model_his' is not defined

## Train Model with seg_length 20

In [32]:
# Model parameters
seg_length = 20
pool_sizes_hori = [2, 2, 2, 2] # sizes of pooling layers (in horizontal direction)

# Load Training Data
X_train = np.load('./dataset/X_train_seg{}.npy'.format(seg_length))
Y_train_pre = np.load('./dataset/Y_train_seg{}.npy'.format(seg_length)).astype(int)
Y_train = np.zeros((Y_train_pre.shape[0], 10))
Y_train[np.arange(Y_train_pre.shape[0]), Y_train_pre] = 1

# Load Test Data
X_test = np.load('./dataset/X_test_seg{}.npy'.format(seg_length))
Y_test_pre = np.load('./dataset/Y_test_seg{}.npy'.format(seg_length)).astype(int)
Y_test = np.zeros((Y_test_pre.shape[0], 10))
Y_test[np.arange(Y_test_pre.shape[0]), Y_test_pre] = 1

# Generate model
model = gen_model_3(seg_length, pool_sizes_hori)

# # Load Model Weights
# model_20.load_weights('./weights/local_cnn_{}.h5'.format(seg_length), by_name=True)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
# Train Model
model_his = model.fit(X_train, Y_train, batch_size=256, validation_data=(X_test, Y_test), nb_epoch=300)

Train on 61200 samples, validate on 6800 samples
Epoch 1/300
61200/61200 [==============================] - 41s - loss: 2.4033 - acc: 0.2521 - val_loss: 1.7151 - val_acc: 0.3913
Epoch 2/300
61200/61200 [==============================] - 41s - loss: 1.7468 - acc: 0.3734 - val_loss: 1.5242 - val_acc: 0.4579
Epoch 3/300
61200/61200 [==============================] - 41s - loss: 1.6032 - acc: 0.4304 - val_loss: 1.3927 - val_acc: 0.4978
Epoch 4/300
61200/61200 [==============================] - 41s - loss: 1.5109 - acc: 0.4652 - val_loss: 1.3977 - val_acc: 0.5006
Epoch 5/300
61200/61200 [==============================] - 41s - loss: 1.4455 - acc: 0.4927 - val_loss: 1.3522 - val_acc: 0.5100
Epoch 6/300
61200/61200 [==============================] - 41s - loss: 1.3991 - acc: 0.5077 - val_loss: 1.2787 - val_acc: 0.5553
Epoch 7/300
61200/61200 [==============================] - 41s - loss: 1.3490 - acc: 0.5284 - val_loss: 1.3169 - val_acc: 0.5318
Epoch 8/300
61200/61200 [=======================

61200/61200 [==============================] - 41s - loss: 0.8217 - acc: 0.7163 - val_loss: 0.9939 - val_acc: 0.6531
Epoch 64/300
61200/61200 [==============================] - 41s - loss: 0.8251 - acc: 0.7146 - val_loss: 0.9630 - val_acc: 0.6622
Epoch 65/300
61200/61200 [==============================] - 41s - loss: 0.8258 - acc: 0.7150 - val_loss: 0.9774 - val_acc: 0.6563
Epoch 66/300
61200/61200 [==============================] - 41s - loss: 0.8179 - acc: 0.7180 - val_loss: 0.9867 - val_acc: 0.6503
Epoch 67/300
61200/61200 [==============================] - 41s - loss: 0.8202 - acc: 0.7155 - val_loss: 0.9887 - val_acc: 0.6609
Epoch 68/300
61200/61200 [==============================] - 41s - loss: 0.8187 - acc: 0.7180 - val_loss: 1.0071 - val_acc: 0.6554
Epoch 69/300
61200/61200 [==============================] - 41s - loss: 0.8191 - acc: 0.7142 - val_loss: 1.0081 - val_acc: 0.6526
Epoch 70/300
61200/61200 [==============================] - 41s - loss: 0.8149 - acc: 0.7184 - val_loss

61200/61200 [==============================] - 41s - loss: 0.7680 - acc: 0.7328 - val_loss: 0.9880 - val_acc: 0.6547
Epoch 126/300
61200/61200 [==============================] - 41s - loss: 0.7620 - acc: 0.7365 - val_loss: 0.9854 - val_acc: 0.6581
Epoch 127/300
61200/61200 [==============================] - 41s - loss: 0.7622 - acc: 0.7353 - val_loss: 1.0034 - val_acc: 0.6550
Epoch 128/300
61200/61200 [==============================] - 41s - loss: 0.7631 - acc: 0.7354 - val_loss: 0.9806 - val_acc: 0.6597
Epoch 129/300
61200/61200 [==============================] - 41s - loss: 0.7633 - acc: 0.7360 - val_loss: 0.9756 - val_acc: 0.6731
Epoch 130/300
61200/61200 [==============================] - 41s - loss: 0.7636 - acc: 0.7346 - val_loss: 1.0197 - val_acc: 0.6493
Epoch 131/300
61200/61200 [==============================] - 41s - loss: 0.7608 - acc: 0.7362 - val_loss: 0.9815 - val_acc: 0.6588
Epoch 132/300
61200/61200 [==============================] - 41s - loss: 0.7631 - acc: 0.7326 - v

61200/61200 [==============================] - 41s - loss: 0.7384 - acc: 0.7435 - val_loss: 0.9973 - val_acc: 0.6612
Epoch 188/300
61200/61200 [==============================] - 41s - loss: 0.7345 - acc: 0.7444 - val_loss: 0.9845 - val_acc: 0.6731
Epoch 189/300
61200/61200 [==============================] - 41s - loss: 0.7402 - acc: 0.7423 - val_loss: 0.9638 - val_acc: 0.6707
Epoch 190/300
61200/61200 [==============================] - 41s - loss: 0.7366 - acc: 0.7448 - val_loss: 1.0309 - val_acc: 0.6509
Epoch 191/300
61200/61200 [==============================] - 41s - loss: 0.7437 - acc: 0.7427 - val_loss: 0.9803 - val_acc: 0.6612
Epoch 192/300
61200/61200 [==============================] - 41s - loss: 0.7422 - acc: 0.7423 - val_loss: 0.9583 - val_acc: 0.6778
Epoch 193/300
61200/61200 [==============================] - 41s - loss: 0.7371 - acc: 0.7438 - val_loss: 0.9704 - val_acc: 0.6679
Epoch 194/300
61200/61200 [==============================] - 41s - loss: 0.7350 - acc: 0.7463 - v

61200/61200 [==============================] - 41s - loss: 0.7208 - acc: 0.7485 - val_loss: 0.9765 - val_acc: 0.6675
Epoch 250/300
61200/61200 [==============================] - 41s - loss: 0.7212 - acc: 0.7489 - val_loss: 1.0042 - val_acc: 0.6584
Epoch 251/300
61200/61200 [==============================] - 41s - loss: 0.7209 - acc: 0.7503 - val_loss: 1.0098 - val_acc: 0.6618
Epoch 252/300
61200/61200 [==============================] - 41s - loss: 0.7165 - acc: 0.7493 - val_loss: 0.9807 - val_acc: 0.6668
Epoch 253/300
61200/61200 [==============================] - 41s - loss: 0.7199 - acc: 0.7517 - val_loss: 0.9595 - val_acc: 0.6703
Epoch 254/300
61200/61200 [==============================] - 41s - loss: 0.7180 - acc: 0.7507 - val_loss: 1.0424 - val_acc: 0.6469
Epoch 255/300
61200/61200 [==============================] - 41s - loss: 0.7259 - acc: 0.7474 - val_loss: 1.0045 - val_acc: 0.6607
Epoch 256/300
61200/61200 [==============================] - 41s - loss: 0.7187 - acc: 0.7514 - v

In [18]:
# Save Model weights
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
model_20.save_weights('./weights/local_cnn_3_{}.h5'.format(seg_length))

NameError: name 'model_20' is not defined

## Train Model with seg_length 30

In [23]:
# Model parameters
seg_length = 30
pool_sizes_hori = [2, 2, 2] # sizes of pooling layers (in horizontal direction)

# Load Training Data
X_train = np.load('./dataset/X_train_seg{}.npy'.format(seg_length))
Y_train_pre = np.load('./dataset/Y_train_seg{}.npy'.format(seg_length)).astype(int)
Y_train = np.zeros((Y_train_pre.shape[0], 10))
Y_train[np.arange(Y_train_pre.shape[0]), Y_train_pre] = 1

# Load Test Data
X_test = np.load('./dataset/X_test_seg{}.npy'.format(seg_length))
Y_test_pre = np.load('./dataset/Y_test_seg{}.npy'.format(seg_length)).astype(int)
Y_test = np.zeros((Y_test_pre.shape[0], 10))
Y_test[np.arange(Y_test_pre.shape[0]), Y_test_pre] = 1

# Generate model
model_30 = gen_model_3(seg_length, pool_sizes_hori)

# # Load Model Weights
# model_30.load_weights('./weights/local_cnn_{}.h5'.format(seg_length), by_name=True)

# Compile model
model_30.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Train Model
model_his = model_30.fit(X_train, Y_train, batch_size=256, validation_data=(X_test, Y_test), nb_epoch=200)

Train on 40500 samples, validate on 4500 samples
Epoch 1/200
40500/40500 [==============================] - 26s - loss: 3.1681 - acc: 0.2565 - val_loss: 1.7385 - val_acc: 0.3811
Epoch 2/200
40500/40500 [==============================] - 26s - loss: 1.7221 - acc: 0.4011 - val_loss: 1.5199 - val_acc: 0.4836
Epoch 3/200
40500/40500 [==============================] - 26s - loss: 1.5230 - acc: 0.4694 - val_loss: 1.4516 - val_acc: 0.4856
Epoch 4/200
40500/40500 [==============================] - 26s - loss: 1.4365 - acc: 0.5038 - val_loss: 1.3918 - val_acc: 0.5142
Epoch 5/200
40500/40500 [==============================] - 25s - loss: 1.3729 - acc: 0.5282 - val_loss: 1.3081 - val_acc: 0.5520
Epoch 6/200
40500/40500 [==============================] - 26s - loss: 1.3203 - acc: 0.5482 - val_loss: 1.2551 - val_acc: 0.5702
Epoch 7/200
40500/40500 [==============================] - 26s - loss: 1.2807 - acc: 0.5607 - val_loss: 1.2270 - val_acc: 0.5816
Epoch 8/200
40500/40500 [=======================

40500/40500 [==============================] - 26s - loss: 0.7512 - acc: 0.7435 - val_loss: 0.9413 - val_acc: 0.6793
Epoch 126/200
40500/40500 [==============================] - 26s - loss: 0.7514 - acc: 0.7424 - val_loss: 0.8861 - val_acc: 0.6953
Epoch 127/200
40500/40500 [==============================] - 26s - loss: 0.7451 - acc: 0.7440 - val_loss: 0.9020 - val_acc: 0.6933
Epoch 128/200
40500/40500 [==============================] - 26s - loss: 0.7448 - acc: 0.7435 - val_loss: 0.8830 - val_acc: 0.6978
Epoch 129/200
40500/40500 [==============================] - 26s - loss: 0.7517 - acc: 0.7419 - val_loss: 0.9031 - val_acc: 0.6889
Epoch 130/200
40500/40500 [==============================] - 26s - loss: 0.7415 - acc: 0.7452 - val_loss: 0.8782 - val_acc: 0.6956
Epoch 131/200
40500/40500 [==============================] - 26s - loss: 0.7451 - acc: 0.7445 - val_loss: 0.9294 - val_acc: 0.6802
Epoch 132/200
40500/40500 [==============================] - 26s - loss: 0.7439 - acc: 0.7469 - v

In [25]:
# Save Model weights
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
model_30.save_weights('./weights/local_cnn_3_{}.h5'.format(seg_length))

## Train Model with seg_length 60

In [26]:
# Model parameters
seg_length = 60
pool_sizes_hori = [3, 3, 2] # sizes of pooling layers (in horizontal direction)

# Load Training Data
X_train = np.load('./dataset/X_train_seg{}.npy'.format(seg_length))
Y_train_pre = np.load('./dataset/Y_train_seg{}.npy'.format(seg_length)).astype(int)
Y_train = np.zeros((Y_train_pre.shape[0], 10))
Y_train[np.arange(Y_train_pre.shape[0]), Y_train_pre] = 1

# Load Test Data
X_test = np.load('./dataset/X_test_seg{}.npy'.format(seg_length))
Y_test_pre = np.load('./dataset/Y_test_seg{}.npy'.format(seg_length)).astype(int)
Y_test = np.zeros((Y_test_pre.shape[0], 10))
Y_test[np.arange(Y_test_pre.shape[0]), Y_test_pre] = 1

# Generate model
model = gen_model_3(seg_length, pool_sizes_hori)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Train Model
model_his = model.fit(X_train, Y_train, batch_size=256, validation_data=(X_test, Y_test), nb_epoch=300)

Train on 19800 samples, validate on 2200 samples
Epoch 1/300
19800/19800 [==============================] - 15s - loss: 4.9614 - acc: 0.2034 - val_loss: 1.9345 - val_acc: 0.3282
Epoch 2/300
19800/19800 [==============================] - 15s - loss: 2.0095 - acc: 0.3196 - val_loss: 1.9145 - val_acc: 0.3400
Epoch 3/300
19800/19800 [==============================] - 15s - loss: 1.7349 - acc: 0.3917 - val_loss: 1.7639 - val_acc: 0.4027
Epoch 4/300
19800/19800 [==============================] - 15s - loss: 1.6017 - acc: 0.4387 - val_loss: 1.6357 - val_acc: 0.4391
Epoch 5/300
19800/19800 [==============================] - 15s - loss: 1.5073 - acc: 0.4727 - val_loss: 1.5695 - val_acc: 0.4582
Epoch 6/300
19800/19800 [==============================] - 15s - loss: 1.4398 - acc: 0.4987 - val_loss: 1.6164 - val_acc: 0.4482
Epoch 7/300
19800/19800 [==============================] - 15s - loss: 1.3775 - acc: 0.5227 - val_loss: 1.5963 - val_acc: 0.4532
Epoch 8/300
19800/19800 [=======================

19800/19800 [==============================] - 15s - loss: 0.5925 - acc: 0.7970 - val_loss: 0.9304 - val_acc: 0.7032
Epoch 126/300
19800/19800 [==============================] - 15s - loss: 0.5953 - acc: 0.7933 - val_loss: 0.8942 - val_acc: 0.7109
Epoch 127/300
19800/19800 [==============================] - 15s - loss: 0.5770 - acc: 0.8027 - val_loss: 0.8975 - val_acc: 0.7068
Epoch 128/300
19800/19800 [==============================] - 15s - loss: 0.5879 - acc: 0.8007 - val_loss: 0.9002 - val_acc: 0.7100
Epoch 129/300
19800/19800 [==============================] - 15s - loss: 0.5962 - acc: 0.7964 - val_loss: 0.8889 - val_acc: 0.7191
Epoch 130/300
19800/19800 [==============================] - 15s - loss: 0.5921 - acc: 0.7973 - val_loss: 0.8886 - val_acc: 0.7218
Epoch 131/300
19800/19800 [==============================] - 15s - loss: 0.5831 - acc: 0.8009 - val_loss: 0.8908 - val_acc: 0.7145
Epoch 132/300
19800/19800 [==============================] - 15s - loss: 0.5825 - acc: 0.8021 - v

19800/19800 [==============================] - 15s - loss: 0.5295 - acc: 0.8164 - val_loss: 0.9021 - val_acc: 0.7191
Epoch 250/300
19800/19800 [==============================] - 15s - loss: 0.5214 - acc: 0.8209 - val_loss: 0.9102 - val_acc: 0.7177
Epoch 251/300
19800/19800 [==============================] - 15s - loss: 0.5364 - acc: 0.8160 - val_loss: 0.9214 - val_acc: 0.7068
Epoch 252/300
19800/19800 [==============================] - 15s - loss: 0.5262 - acc: 0.8196 - val_loss: 0.9898 - val_acc: 0.6973
Epoch 253/300
19800/19800 [==============================] - 15s - loss: 0.5294 - acc: 0.8200 - val_loss: 0.9442 - val_acc: 0.7027
Epoch 254/300
19800/19800 [==============================] - 15s - loss: 0.5275 - acc: 0.8190 - val_loss: 0.9355 - val_acc: 0.7095
Epoch 255/300
19800/19800 [==============================] - 15s - loss: 0.5222 - acc: 0.8203 - val_loss: 0.8964 - val_acc: 0.7182
Epoch 256/300
19800/19800 [==============================] - 15s - loss: 0.5273 - acc: 0.8193 - v

In [32]:
# Save Model weights
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
model.save_weights('./weights/local_cnn_3_{}.h5'.format(seg_length))

## Train Model with seg_length 120

In [16]:
# Model parameters
seg_length = 120
pool_sizes_hori = [3, 3, 2, 2] # sizes of pooling layers (in horizontal direction)

# Load Training Data
X_train = np.load('./dataset/X_train_seg{}.npy'.format(seg_length))
Y_train_pre = np.load('./dataset/Y_train_seg{}.npy'.format(seg_length)).astype(int)
Y_train = np.zeros((Y_train_pre.shape[0], 10))
Y_train[np.arange(Y_train_pre.shape[0]), Y_train_pre] = 1

# Load Test Data
X_test = np.load('./dataset/X_test_seg{}.npy'.format(seg_length))
Y_test_pre = np.load('./dataset/Y_test_seg{}.npy'.format(seg_length)).astype(int)
Y_test = np.zeros((Y_test_pre.shape[0], 10))
Y_test[np.arange(Y_test_pre.shape[0]), Y_test_pre] = 1

# Training Data Augmentation
# datagen.fit(X_train)

# Generate model
model = gen_model_3(seg_length, pool_sizes_hori)

# Load Model
# model = load_model('./models/cnn_{}.h5'.format(seg_length))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Train Model
# model_his = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=256),
#                     samples_per_epoch=len(X_train), validation_data=(X_test, Y_test), nb_epoch=200)
model_his = model.fit(X_train, Y_train, batch_size=256, validation_data=(X_test, Y_test), nb_epoch=400)

Train on 9900 samples, validate on 1100 samples
Epoch 1/400
9900/9900 [==============================] - 12s - loss: 5.4039 - acc: 0.1336 - val_loss: 2.2894 - val_acc: 0.1536
Epoch 2/400
9900/9900 [==============================] - 12s - loss: 2.2873 - acc: 0.1052 - val_loss: 2.1977 - val_acc: 0.1955
Epoch 3/400
9900/9900 [==============================] - 12s - loss: 2.1812 - acc: 0.1890 - val_loss: 2.0098 - val_acc: 0.2182
Epoch 4/400
9900/9900 [==============================] - 12s - loss: 2.0795 - acc: 0.2176 - val_loss: 1.9151 - val_acc: 0.2845
Epoch 5/400
9900/9900 [==============================] - 12s - loss: 1.9986 - acc: 0.2356 - val_loss: 1.8667 - val_acc: 0.2991
Epoch 6/400
9900/9900 [==============================] - 12s - loss: 1.9475 - acc: 0.2607 - val_loss: 1.8037 - val_acc: 0.3145
Epoch 7/400
9900/9900 [==============================] - 12s - loss: 1.9074 - acc: 0.2716 - val_loss: 1.7230 - val_acc: 0.3809
Epoch 8/400
9900/9900 [==============================] - 12s - 

9900/9900 [==============================] - 12s - loss: 0.3505 - acc: 0.8730 - val_loss: 1.2535 - val_acc: 0.7073
Epoch 128/400
9900/9900 [==============================] - 12s - loss: 0.3505 - acc: 0.8760 - val_loss: 1.1501 - val_acc: 0.7127
Epoch 129/400
9900/9900 [==============================] - 12s - loss: 0.3487 - acc: 0.8800 - val_loss: 1.1267 - val_acc: 0.7218
Epoch 130/400
9900/9900 [==============================] - 12s - loss: 0.3419 - acc: 0.8761 - val_loss: 1.1314 - val_acc: 0.7100
Epoch 131/400
9900/9900 [==============================] - 12s - loss: 0.3404 - acc: 0.8798 - val_loss: 1.1005 - val_acc: 0.7255
Epoch 132/400
9900/9900 [==============================] - 12s - loss: 0.3365 - acc: 0.8822 - val_loss: 1.1867 - val_acc: 0.7064
Epoch 133/400
9900/9900 [==============================] - 12s - loss: 0.3331 - acc: 0.8823 - val_loss: 1.1218 - val_acc: 0.7227
Epoch 134/400
9900/9900 [==============================] - 12s - loss: 0.3266 - acc: 0.8864 - val_loss: 1.0701 

9900/9900 [==============================] - 12s - loss: 0.1874 - acc: 0.9356 - val_loss: 1.5427 - val_acc: 0.7145
Epoch 254/400
9900/9900 [==============================] - 12s - loss: 0.1913 - acc: 0.9331 - val_loss: 1.4147 - val_acc: 0.7282
Epoch 255/400
9900/9900 [==============================] - 12s - loss: 0.1848 - acc: 0.9343 - val_loss: 1.4139 - val_acc: 0.7255
Epoch 256/400
9900/9900 [==============================] - 12s - loss: 0.2064 - acc: 0.9265 - val_loss: 1.6161 - val_acc: 0.6927
Epoch 257/400
9900/9900 [==============================] - 12s - loss: 0.1892 - acc: 0.9328 - val_loss: 1.5308 - val_acc: 0.7145
Epoch 258/400
9900/9900 [==============================] - 12s - loss: 0.1862 - acc: 0.9334 - val_loss: 1.5223 - val_acc: 0.7191
Epoch 259/400
9900/9900 [==============================] - 12s - loss: 0.1924 - acc: 0.9305 - val_loss: 1.5822 - val_acc: 0.7009
Epoch 260/400
9900/9900 [==============================] - 12s - loss: 0.1976 - acc: 0.9303 - val_loss: 1.5048 

9900/9900 [==============================] - 12s - loss: 0.1443 - acc: 0.9523 - val_loss: 1.6634 - val_acc: 0.7182
Epoch 380/400
9900/9900 [==============================] - 12s - loss: 0.1583 - acc: 0.9451 - val_loss: 1.6632 - val_acc: 0.7282
Epoch 381/400
9900/9900 [==============================] - 12s - loss: 0.1550 - acc: 0.9503 - val_loss: 1.6683 - val_acc: 0.7127
Epoch 382/400
9900/9900 [==============================] - 12s - loss: 0.1578 - acc: 0.9449 - val_loss: 1.5593 - val_acc: 0.7300
Epoch 383/400
9900/9900 [==============================] - 12s - loss: 0.1729 - acc: 0.9412 - val_loss: 1.7241 - val_acc: 0.7155
Epoch 384/400
9900/9900 [==============================] - 12s - loss: 0.1653 - acc: 0.9432 - val_loss: 1.6324 - val_acc: 0.7191
Epoch 385/400
9900/9900 [==============================] - 12s - loss: 0.1491 - acc: 0.9491 - val_loss: 1.6813 - val_acc: 0.7191
Epoch 386/400
9900/9900 [==============================] - 12s - loss: 0.1578 - acc: 0.9480 - val_loss: 1.5311 

## Train Model with seg_length 240

In [23]:
# Model parameters
seg_length = 240
pool_sizes_hori = [4, 4, 3, 2] # sizes of pooling layers (in horizontal direction)

X = np.load('./dataset/X_train_seg{}.npy'.format(seg_length))
Y_pre = np.load('./dataset/Y_train_seg{}.npy'.format(seg_length)).astype(int)

# Encode Y_pre to one-hot(Y)
Y = np.zeros((Y_pre.shape[0], 10))
Y[np.arange(Y_pre.shape[0]), Y_pre] = 1

# Generate model
model = gen_model_3(seg_length, pool_sizes_hori)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Split Dataset (90% train + 10% dev)
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.1)

In [29]:
# Train Model
model_his = model.fit(X_train, Y_train, batch_size=256, validation_data=(X_dev, Y_dev), nb_epoch=100)

Train on 4050 samples, validate on 450 samples
Epoch 1/100
4050/4050 [==============================] - 7s - loss: 0.1370 - acc: 0.9533 - val_loss: 0.8963 - val_acc: 0.8111
Epoch 2/100
4050/4050 [==============================] - 7s - loss: 0.1305 - acc: 0.9521 - val_loss: 0.8664 - val_acc: 0.8222
Epoch 3/100
4050/4050 [==============================] - 7s - loss: 0.1263 - acc: 0.9536 - val_loss: 0.8026 - val_acc: 0.8267
Epoch 4/100
4050/4050 [==============================] - 7s - loss: 0.1325 - acc: 0.9548 - val_loss: 0.8261 - val_acc: 0.8267
Epoch 5/100
4050/4050 [==============================] - 7s - loss: 0.1334 - acc: 0.9551 - val_loss: 0.7378 - val_acc: 0.8422
Epoch 6/100
4050/4050 [==============================] - 7s - loss: 0.1283 - acc: 0.9560 - val_loss: 0.8330 - val_acc: 0.8200
Epoch 7/100
4050/4050 [==============================] - 7s - loss: 0.1505 - acc: 0.9432 - val_loss: 0.7398 - val_acc: 0.8333
Epoch 8/100
4050/4050 [==============================] - 7s - loss: 0.1

In [16]:
# Save Model weights
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
model.save_weights('./weights/local_cnn_{}.h5'.format(seg_length))